# Chains in LangChain

In [2]:
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel , RunnableBranch , RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

load_dotenv()

# Pydantic Schema
class Feedback(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(description="Give the sentiment of the Feedback")

# Models and Parsers
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
parser = StrOutputParser()
parser2 = PydanticOutputParser(pydantic_object=Feedback)

# Prompt Template with format instructions
prompt1 = PromptTemplate(
    template="Classify the sentiment of the following feedback text into positive or negative:\n"
             "{feedback}\n\n{format_instructions}",
    input_variables=['feedback'],
    partial_variables={'format_instructions': parser2.get_format_instructions()}
)
print(prompt1.format(feedback="This is terrible phone"))


Classify the sentiment of the following feedback text into positive or negative:
This is terrible phone

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sentiment": {"description": "Give the sentiment of the Feedback", "enum": ["positive", "negative"], "title": "Sentiment", "type": "string"}}, "required": ["sentiment"]}
```


In [3]:
# Chain (using PydanticOutputParser)
classifier_chain = prompt1 | model | parser2
result1 = classifier_chain.invoke({'feedback':'This is terrible phone'})
result1 

Feedback(sentiment='negative')

In [3]:
prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback'] ) 

In [4]:
branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

In [5]:
chain = classifier_chain | branch_chain 

result = chain.invoke({'feedback':'This is terrible phone'})
print(result)
chain.get_graph().print_ascii()

When responding to negative feedback, the key is to be empathetic, apologetic, and proactive. Since the specific content of the feedback isn't provided, I'll give you a few adaptable templates.

**Key Elements of a Good Response to Negative Feedback:**

1.  **Thank them for the feedback:** Even negative feedback is valuable for improvement.
2.  **Acknowledge and Empathize:** Show you understand their frustration or disappointment.
3.  **Apologize Sincerely:** Take responsibility, even if it's just for their negative experience.
4.  **Validate their experience:** Reassure them that their feelings are understood and that this is not the standard you aim for.
5.  **Offer a Solution or Path to Resolution:** This is crucial. How will you make it right?
6.  **Commit to Improvement:** Show that you're taking their feedback seriously and will use it to get better.
7.  **Invite Further Communication:** Provide a direct way for them to follow up.

---

**Option 1: General Response (when you need

# manually Running

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv

load_dotenv()

# -------------------------
# 1️⃣ Pydantic Schema
# -------------------------
class Feedback(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(
        description="Give the sentiment of the Feedback"
    )

# -------------------------
# 2️⃣ Model and Parsers
# -------------------------
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
parser = StrOutputParser()
parser2 = PydanticOutputParser(pydantic_object=Feedback)

# -------------------------
# 3️⃣ Classification Prompt
# -------------------------
prompt1 = PromptTemplate(
    template="Classify the sentiment of the following feedback text into positive or negative:\n"
             "{feedback}\n\n{format_instructions}",
    input_variables=['feedback'],
    partial_variables={'format_instructions': parser2.get_format_instructions()}
)

# -------------------------
# 4️⃣ Response Prompts
# -------------------------
prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)


In [7]:
# -------------------------
# 5️⃣ Manual Execution
# -------------------------
feedback_text = "This is terrible phone"

# Step 1: Classify
formatted_prompt = prompt1.format(feedback=feedback_text) # prompt1 input
classification_raw = model.invoke(formatted_prompt) # model execution
classification = parser2.parse(classification_raw.content) # convert output to pydantic object

print("Sentiment:", classification.sentiment)

# Step 2: Branch manually
if classification.sentiment == "positive":
    formatted_response_prompt = prompt2.format(feedback=feedback_text)
elif classification.sentiment == "negative":
    formatted_response_prompt = prompt3.format(feedback=feedback_text)
else:
    print("Could not find sentiment")
    exit()

# Step 3: Generate response
response_raw = model.invoke(formatted_response_prompt)
final_response = parser.parse(response_raw.content)


Sentiment: negative
